In [ ]:
# Unzip the downloaded dataset
!unzip Soil_classification_dataset_processed_class3.zip -d /content/Soil-classification-dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/Soil-classification-dataset/Soil_classification_dataset_processed_class3/Moderate_moisture/soil_moisture_india_and_sri_lanka_2015-04-08_class_3_patch_36.png  
  inflating: /content/Soil-classification-dataset/Soil_classification_dataset_processed_class3/Moderate_moisture/soil_moisture_india_and_sri_lanka_2015-04-11_class_3_patch_25.png  
  inflating: /content/Soil-classification-dataset/Soil_classification_dataset_processed_class3/Moderate_moisture/soil_moisture_india_and_sri_lanka_2015-04-11_class_3_patch_36.png  
  inflating: /content/Soil-classification-dataset/Soil_classification_dataset_processed_class3/Moderate_moisture/soil_moisture_india_and_sri_lanka_2015-04-11_class_3_patch_51.png  
  inflating: /content/Soil-classification-dataset/Soil_classification_dataset_processed_class3/Moderate_moisture/soil_moisture_india_and_sri_lanka_2015-04-11_class_3_patch_79.png  
  inflating: /content/Soil-classification-da

In [ ]:
import os
import cv2
import numpy as np

def preprocess_image(image_path, replacement_color):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Define white color threshold
    lower_white = np.array([251, 251, 251])  # Slightly less than pure white
    upper_white = np.array([255, 255, 255])  # Pure white

    # Create mask for white pixels
    white_mask = cv2.inRange(image, lower_white, upper_white)

    """# Replace white pixels with the specified replacement color
    white_pixels = np.column_stack(np.where(white_mask > 0))
    for (y, x) in white_pixels:
        image[y, x] = replacement_color"""

    # Replace white pixels with the replacement color
    image[white_mask > 0] = replacement_color

    return image

# Directory containing images
image_path = "/content/Soil-classification-dataset/Soil_classification_dataset_processed_class3"

# Output directory to save processed images
output_path = "/content/Processed_Soil_Classification2"
os.makedirs(output_path, exist_ok=True)

# Define replacement colors for each class
class_colors = {
    "Dry": [252, 98, 56],  # fc6238
    "Moderate_moisture": [141, 215, 191],  # 8dd7bf
    "Highest_moisture": [0, 101, 162]  # 0065a2
}

# Process images in all subfolders and maintain folder structure
for root, dirs, files in os.walk(image_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):  # Supported image formats
            input_file = os.path.join(root, file)

            # Get class name (subfolder name)
            class_name = os.path.basename(root)
            if class_name not in class_colors:
                continue  # Skip folders without a defined replacement color

            # Get the replacement color for the class
            replacement_color = class_colors[class_name]

            # Create corresponding output directory
            relative_path = os.path.relpath(root, image_path)  # Get relative subfolder structure
            output_dir = os.path.join(output_path, relative_path)
            os.makedirs(output_dir, exist_ok=True)

            # Define output file path
            output_file = os.path.join(output_dir, file)

            # Preprocess the image
            processed_image = preprocess_image(input_file, replacement_color)

            # Convert to BGR and save using OpenCV
            processed_image = cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_file, processed_image)

print("Processing completed. Processed images saved in:", output_path)


Processing completed. Processed images saved in: /content/Processed_Soil_Classification2


In [ ]:
import shutil

# Define the folder you want to zip and the name of the output zip file
folder_to_zip = '/content/Processed_Soil_Classification2'  # Replace with your folder path
output_zip_file = '/content/Processed_Soil_Classification.zip'  # Replace with the desired output zip file path

# Use shutil to create a zip file
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)

print(f'Folder {folder_to_zip} has been zipped into {output_zip_file}')

Folder /content/Processed_Soil_Classification2 has been zipped into /content/Processed_Soil_Classification.zip
